In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload


import holodeck as holo
from holodeck import single_sources, utils, plot, detstats, anisotropy
from holodeck.constants import YR

import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
import h5py



## Load and concatenate pixel strains and harmonic coefficients for NSIDE 32

In [ ]:
NBEST = 1

file1 = np.load('/Users/emigardiner/GWs/holodeck/output/brc_output/ss51-2023-05-22_uniform_07a_n1000_r100_f40_l2000/anisotropy/sph_harm_hc2dOm_lmax6_ns32_r50_b00-49.npz')
print(f"{file1.files=}")
# load ss info
shape = file1['ss_shape']
nsamps, nfreqs, nreals, nloudest = shape[0], shape[1], shape[2], shape[3]
fobs = file1['fobs']

# load ranking info
nsort = file1['nsort'][:NBEST]
fidx = file1['fidx']
hc_ref = file1['hc_ref']

# load harmonics info
nside = file1['nside']
lmax  = file1['lmax']

# load map and harmonics from 1st split chunk
moll_hc_best1 = file1['moll_hc_best'][:NBEST]
Cl_best1 = file1['Cl_best'][:NBEST]
nbest1 = len(moll_hc_best1)

file1.close()



In [ ]:
print(fobs[1]*10**9)

In [ ]:
moll_hc = moll_hc_best1[0]
print(f"{moll_hc.shape}")

In [ ]:
ff=1
rr=3
fig, ax = plot.figax()
hp.mollview(moll_hc[ff,rr], hold=True, title='Simulated %.2f yr$^{-1}$ GW Sky' % ( fobs[ff]*YR), 
            unit='$h_c^2/\mathrm{sr}^2$', min = 4e-33, max=1e-30,
            
            )
# hp.graticule(alpha=0.5)
fig.tight_layout()

In [ ]:
ff=1
rr=3
fig,ax = plot.figax()

smooth_map = hp.smoothing(moll_hc[ff,rr], fwhm=np.radians(1.) )


# mpl.rcParams.update({'font.size': 14}) 
hp.mollview(smooth_map, hold=True, title=None, fig=fig,
            unit='$h_c^2($f$=%.2f\ \mathrm{yr}^{-1})$'  % ( fobs[ff]*YR), 
            min = 4e-33, max=8e-31, cbar=False
            )
# hp.graticule(alpha=0.5)

fig, ax = plt.gcf(), plt.gca()
image = ax.get_images()[0]

# fig.colorbar(image, orientation='horizontal', 
#              label='$h_c^2 (f=%0.2f\ \mathrm{yr^{-1}})$' % (fobs[ff]*YR),
#              pad=0.02, shrink=0.6, aspect=30)
# fig.colorbar()
fig.tight_layout()

In [ ]:

ff=1
rr=3
lmax=6
fig,ax = plot.figax()


Cl = hp.anafast(moll_hc[ff,rr], lmax=lmax)
Cl_map = hp.synfast(Cl, nside, lmax=lmax)

seed = np.random.randint(99999)   # get a random number
print(seed)                       # print it out so we can reuse it if desired
seed = 34891
np.random.seed(seed)   

# mpl.rcParams.update({'font.size': 14}) 
hp.mollview(Cl_map, hold=True, title='Reconstructed with $\ell_\mathrm{max}=%d$' % lmax, fig=fig,
            unit='$h_c^2($f$=%.2f\ \mathrm{yr}^{-1})$'  % ( fobs[ff]*YR), 
            cbar=False, #min = 4e-33, max=8e-31, 
            )
# hp.graticule(alpha=0.5)

fig, ax = plt.gcf(), plt.gca()
image = ax.get_images()[0]

# fig.colorbar(image, orientation='horizontal', 
#              label='$h_c^2 (f=%0.2f\ \mathrm{yr^{-1}})$' % (fobs[ff]*YR),
#              pad=0.02, shrink=0.6, aspect=30)
# fig.colorbar()
fig.tight_layout()

In [ ]:

ff=1
rr=3
lmax=3


Cl = hp.anafast(moll_hc[ff,rr], lmax=lmax)

seed = 50158   # get a random number
print(seed)                       # print it out so we can reuse it if desired
np.random.seed(seed)   

fig,ax = plot.figax()


Cl_map = hp.synfast(Cl, nside, lmax=lmax)
# mpl.rcParams.update({'font.size': 14}) 
hp.mollview(Cl_map, hold=True, 
            title='Reconstructed with $\ell_\mathrm{max}=%d$, seed=%d' % (lmax, seed),
            cbar=False
            #min = 4e-33, max=1.5e-31, cbar=False
            )
fig, ax = plt.gcf(), plt.gca()

In [ ]:

ff=1
rr=3
lmax=3


Cl = hp.anafast(moll_hc[ff,rr], lmax=lmax)

for ii in range(20):
    seed = np.random.randint(99999)   # get a random number
    print(seed)                       # print it out so we can reuse it if desired
    np.random.seed(seed)   

    fig,ax = plot.figax()


    Cl_map = hp.synfast(Cl, nside, lmax=lmax)
    # mpl.rcParams.update({'font.size': 14}) 
    hp.mollview(Cl_map, hold=True, 
                title='Reconstructed with $\ell_\mathrm{max}=%d$, seed=%d' % (lmax, seed),
                cbar=False
                #min = 4e-33, max=1.5e-31, cbar=False
                )
    fig, ax = plt.gcf(), plt.gca()
    # fig.savefig(('/Users/emigardiner/GWs/holodeck/ecg-notebooks/anisotropy/mollmaps/log_r%df%d_%02ddeg.png'
    #               % (rr,ff,deg)))
    # plt.close(fig)

    # seed = np.random.randint(99999)   # get a random number
    # print(seed)                       # print it out so we can reuse it if desired
    # np.random.seed(seed)   
    # # mpl.rcParams.update({'font.size': 14}) 
    # hp.mollview(Cl_map, hold=False, title=None, 
    #             unit='$h_c^2($f$=%.2f\ \mathrm{yr}^{-1})$'  % ( fobs[ff]*YR), )
    # fig, ax = plt.gcf(), plt.gca()
    # ax.set_title(', fontsize=12)
    # # fig.colorbar()
    # fig.tight_layout()

In [ ]:
for deg in np.geomspace(5,180,5):
    smooth_map = hp.smoothing(moll_hc[ff,rr], fwhm=np.radians(deg) )
    fig,ax = plot.figax()

    # mpl.rcParams.update({'font.size': 14}) 
    hp.mollview(smooth_map, hold=True, 
                title='Smoothed with fwhm=%d$\deg$' % (deg),
                cbar=False
                #min = 4e-33, max=1.5e-31, cbar=False
                )
    fig, ax = plt.gcf(), plt.gca()
    # fig.savefig(('/Users/emigardiner/GWs/holodeck/ecg-notebooks/anisotropy/mollmaps/log_r%df%d_%02ddeg.png'
    #               % (rr,ff,deg)))
    # plt.close(fig)


# Save varying smoothing

In [ ]:
ff=1
rr=3
# fig,ax = plot.figax()


# deg=30.
ii=0
for deg in np.geomspace(180,.01,100):
    ii+=1
    smooth_map = hp.smoothing(moll_hc[ff,rr], fwhm=np.radians(deg) )
    fig,ax = plot.figax()

    # mpl.rcParams.update({'font.size': 14}) 
    hp.mollview(smooth_map, hold=True, 
                title='Smoothed with fwhm=%d$\deg$' % (deg),
                cbar=False
                #min = 4e-33, max=1.5e-31, cbar=False
                )
    fig, ax = plt.gcf(), plt.gca()
    fig.savefig(('/Users/emigardiner/GWs/holodeck/ecg-notebooks/anisotropy/mollmaps/log180_ii%d_r%df%d_%03ddeg.png'
                  % (ii,rr,ff,deg)))
    plt.close(fig)


In [ ]:
ff=1
rr=3
fig,ax = plot.figax()

smooth_map = hp.smoothing(moll_hc[ff,rr], fwhm=np.radians(60.) )


# mpl.rcParams.update({'font.size': 14}) 
hp.mollview(smooth_map, hold=True, title=None, fig=fig,
            unit='$h_c^2($f$=%.2f\ \mathrm{yr}^{-1})$'  % ( fobs[ff]*YR), 
            cbar=False
            #min = 4e-33, max=1.5e-31, cbar=False
            )
# hp.graticule(alpha=0.5)

fig, ax = plt.gcf(), plt.gca()
image = ax.get_images()[0]

# fig.colorbar(image, orientation='horizontal', 
#              label='$h_c^2 (f=%0.2f\ \mathrm{yr^{-1}})$' % (fobs[ff]*YR),
#              pad=0.02, shrink=0.6, aspect=30)
# fig.colorbar()
fig.tight_layout()

# Find Best Sample

In [ ]:
NBEST = 1

file1 = np.load('/Users/emigardiner/GWs/holodeck/output/brc_output/ss51-2023-05-22_uniform_07a_n1000_r100_f40_l2000/anisotropy/sph_harm_hc2dOm_lmax6_ns32_r50_b00-49.npz')
print(f"{file1.files=}")
# load ranking info
nsort = file1['nsort'][:NBEST]
print(nsort[0])
nn = nsort[0]


file1.close()



# Higher res

In [ ]:
nn=320
ff=1
rr=3

sam_dir = '/Users/emigardiner/GWs/holodeck/output/brc_output/ss51-2023-05-22_uniform_07a_n1000_r100_f40_l2000/'
hdf_name = sam_dir + '/sam_lib.hdf5'
hdf_file = h5py.File(hdf_name, 'r')

hc_ss = hdf_file['hc_ss'][nn,0:5,0:5,:]
hc_bg = hdf_file['hc_bg'][nn,0:5,0:5]
fobs = hdf_file['fobs'][:]

hdf_file.close()


In [ ]:
print(hc_ss.shape)
print(hc_bg.shape)

# make path

In [ ]:
out_path = '/Users/emigardiner/GWs/holodeck/output/mollmaps'
out_dir = out_path+'/09B_nside128'
import os
if (os.path.exists(out_dir) is False):
    print('Making output directory.')
    os.makedirs(out_dir)
else:
    print('Writing to an existing directory.')

make mollmap

In [ ]:
moll_hc = anisotropy.healpix_map(hc_ss, hc_bg, nside=128)
ff=1
rr=3


plot, as is

In [ ]:
fig,ax = plot.figax()

# mpl.rcParams.update({'font.size': 14}) 
hp.mollview(moll_hc[ff,rr], hold=True, 
            title=None,
            cbar=False
            #min = 4e-33, max=1.5e-31, cbar=False
            )
fig, ax = plt.gcf(), plt.gca()
# plt.close(fig)


plot a few smoothing cases

In [ ]:
# deg=30.
ii=0
for deg in np.geomspace(180,1,10):
    ii+=1
    smooth_map = hp.smoothing(moll_hc[ff,rr], fwhm=np.radians(deg) )
    fig,ax = plot.figax()

    # mpl.rcParams.update({'font.size': 14}) 
    hp.mollview(smooth_map, hold=True, 
                title='Smoothed with fwhm=%.1f$\deg$' % (deg),
                cbar=False
                #min = 4e-33, max=1.5e-31, cbar=False
                )
    fig, ax = plt.gcf(), plt.gca()
    # fig.savefig(('/Users/emigardiner/GWs/holodeck/ecg-notebooks/anisotropy/mollmaps/log180_ii%d_r%df%d_%03ddeg.png'
    #               % (ii,rr,ff,deg)))
    # plt.close(fig)


plot and save many for gif

In [ ]:

ii=0
for deg in np.geomspace(180,1,500):
    ii+=1
    smooth_map = hp.smoothing(moll_hc[ff,rr], fwhm=np.radians(deg) )
    fig,ax = plot.figax()

    # mpl.rcParams.update({'font.size': 14}) 
    hp.mollview(smooth_map, hold=True, 
                title=None,
                cbar=False
                #min = 4e-33, max=1.5e-31, cbar=False
                )
    fig, ax = plt.gcf(), plt.gca()
    fig.savefig((out_dir+'/ii%03d_rr%02dff%02d_%03ddeg.png'
                  % (ii,rr,ff,deg)))
    plt.close(fig)
